In [179]:
import pandas as pd
import numpy as np


#longterm = 'GHA_2008_MGFERE_v01_M_Stata8/r7tomerge.dta'
#all_data = 'GHA_2008_MGFERE_v01_M_Stata8/ReplicationDataGhanaJDE.dta'

# obtained by running the first three stata commends in "Table 5: Longer-term results" from JDEreplicationfilesGhana.do
my_data = 'GHA_2008_MGFERE_v01_M_Stata8/mydata.dta' 

#data_longterm = pd.read_stata(longterm)
#data_all = pd.read_stata(all_data)
mydata = pd.read_stata(my_data)

/Users/liujizhou/opt/anaconda3/lib/python3.8/site-packages/pandas/io/stata.py:1433: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [169]:
df_mydata = mydata[mydata.wave==7][['realfinalprofit', 'atreatcash', 'atreatequip', 'wave', 'groupnum' ]]
dummies = pd.get_dummies(df_mydata.groupnum)
df = pd.concat([df_mydata, dummies], axis=1, join='inner')

In [170]:
import statsmodels.api as sm

Y = df.realfinalprofit
X = df[['atreatcash', 'atreatequip']+list(dummies.columns)]
idx = (1-np.isnan(Y))==1
Y = Y[idx]
X = X[idx]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.params[:3])
print(results.tvalues[:3])

const          126.994885
atreatcash      22.557150
atreatequip     43.511862
dtype: float64
const          7.958574
atreatcash     0.855145
atreatequip    1.663011
dtype: float64


In [171]:
Y = df.realfinalprofit
X = df[['atreatcash', 'atreatequip']+list(dummies.columns)]
X = sm.add_constant(X)
Yhat = results.predict(X)
idx = np.isnan(Y)
Y[idx] = Yhat[idx]
model = sm.OLS(Y,X)
results = model.fit()
print(results.params[:3])
print(results.tvalues[:3])

const          126.994885
atreatcash      22.557150
atreatequip     43.511862
dtype: float64
const          13.426499
atreatcash      1.371273
atreatequip     2.645769
dtype: float64


<ipython-input-171-8b04d9b1a7ef>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[idx] = Yhat[idx]


In [172]:
df_mydata['Yhat'] = Y
bad_groups = [991,992,993,994]
df_clean = df_mydata[~df_mydata['groupnum'].isin(bad_groups)]
treatment = np.zeros(len(df_clean))
treatment[df_clean['atreatcash']==1] = 1
treatment[df_clean['atreatequip']==1] = 2
df_clean['treatment'] = treatment
df_sort = df_clean.sort_values(by=['groupnum','treatment'], ascending=True)


dummies = pd.get_dummies(df_sort.groupnum)
df_sort = pd.concat([df_sort, dummies], axis=1, join='inner')
Y = df_sort.Yhat
X = df_sort[['atreatcash', 'atreatequip']+list(dummies.columns)]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.params[:3])
print(results.tvalues[:3])



const          127.875633
atreatcash      21.457336
atreatequip     37.417927
dtype: float64
const          13.557263
atreatcash      1.306567
atreatequip     2.278429
dtype: float64


<ipython-input-172-ec6db9acc87d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['treatment'] = treatment


In [173]:
df_strata = pd.read_stata('GHA_2008_MGFERE_v01_M_Stata8/ReplicationDataGhanaJDE.dta')
df_strata = df_strata[df_strata.wave==2][['groupnum','male_male', 'female_female', 'male_mixed', 'female_mixed',
                                          'highcapture','highcapital']]
df_strata['strata'] = df_strata.male_male*100000 + df_strata.female_female*10000 + df_strata.male_mixed*1000 \
    + df_strata.female_mixed*100 + df_strata.highcapture*10 + df_strata.highcapital

df_strata_sort = df_strata.sort_values(by=['strata','groupnum'], ascending=True)
df_total = df_sort.merge(df_strata_sort.iloc[::4], how='left',on='groupnum')
df_total = df_total.sort_values(by=['strata','groupnum'], ascending=True)

Y = df_total.Yhat
X = df_total[['atreatcash', 'atreatequip']+list(dummies.columns)]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.params[:3])
print(results.tvalues[:3])

const          127.875633
atreatcash      21.457336
atreatequip     37.417927
dtype: float64
const          13.557263
atreatcash      1.306567
atreatequip     2.278429
dtype: float64


In [174]:
df_process_tuple = df_total
df_repeat = df_process_tuple
counts = 1
for i in range(1,len(df_process_tuple)):
    if df_process_tuple.strata[i] != df_process_tuple.strata[i-1]:
        if (counts/4) % 2 ==1:
            repeat = df_process_tuple.iloc[i-8:i-4]
            df_repeat = df_repeat.append(repeat)
        counts = 1
    else:
        counts += 1
df_repeat = df_repeat.sort_values(by=['strata','groupnum'], ascending=True)

Y = df_repeat.Yhat
X = df_repeat[['atreatcash', 'atreatequip']+list(dummies.columns)]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.params[:3])
print(results.tvalues[:3])

const          127.610526
atreatcash      22.557496
atreatequip     37.383746
dtype: float64
const          14.877883
atreatcash      1.528085
atreatequip     2.532442
dtype: float64


In [175]:
from empirical import Inference

df_repeat = df_repeat[['Yhat','treatment','groupnum','strata']]
Ys = df_total.Yhat.to_numpy()
Yr = df_repeat.Yhat.to_numpy()
Ys = Ys.reshape(-1,4)
Yr = Yr.reshape(-1,4)
inf = Inference(Ys, Yr)
print(inf.tau1, inf.tau2)
print(inf.tau1/inf.se_tau1, inf.tau2/inf.se_tau2)

21.457336246269733 37.41792732346764
0.9425579152094034 2.3275920806865464


In [176]:

df_drop = df_process_tuple
counts = 1
for i in range(1,len(df_process_tuple)):
    if df_process_tuple.strata[i] != df_process_tuple.strata[i-1]:
        if (counts/4) % 2 ==1:
             df_drop = df_drop[df_drop.groupnum!=df_process_tuple.groupnum[i-1]]
        counts = 1
    else:
        counts += 1
#df_drop = df_drop.sort_values(by=['groupnum','strata'], ascending=True)

df_drop = df_drop[['Yhat','treatment','groupnum','strata']]
Ys = df_total.Yhat.to_numpy()
Yr = df_drop.Yhat.to_numpy()
Ys = Ys.reshape(-1,4)
Yr = Yr.reshape(-1,4)
inf = Inference(Ys, Yr)
print(inf.tau1, inf.tau2)
print(inf.tau1/inf.se_tau1, inf.tau2/inf.se_tau2)

21.457336246269733 37.41792732346764
0.9519758859594617 2.3269667802599914


In [177]:
Ys_control = Ys[:,:2]
tmp = pd.DataFrame(Ys_control)
Ys_control = tmp.apply(lambda x:np.random.shuffle(x) or x, axis=1).to_numpy()
Ys[:,:2] = Ys_control
inf = Inference(Ys, Ys)
print(inf.tau1, inf.tau2)
print(inf.tau1/inf.se_tau1, inf.tau2/inf.se_tau2)

21.457336246269733 37.41792732346764
0.9616875998688005 2.4068449884923915


In [178]:
Y = df_total.realfinalprofit.to_numpy()
X = df_total[['atreatcash', 'atreatequip']+list(dummies.columns)]
idx = (1-np.isnan(Y))==1
Y = Y[idx]
X = X[idx]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.params[:3])
print(results.tvalues[:3])

Ynan = df_total.realfinalprofit.to_numpy()
Ynan = Ynan.reshape((-1,4))
inf = Inference(Ynan, Ynan)
print(inf.estimator_nan())
print(inf.inference_nan())
print(inf.estimator_nan()[0]/inf.inference_nan()[0], inf.estimator_nan()[1]/inf.inference_nan()[1])

const          129.091394
atreatcash      20.445134
atreatequip     34.044968
dtype: float64
const          8.105086
atreatcash     0.773822
atreatequip    1.301254
dtype: float64
(18.022939021147835, 31.586359921435644)
(26.06917183794431, 19.47102382758535)
0.6913506548341904 1.6222238851500985


TO DO: Repeat and Drop for nan